<a href="https://colab.research.google.com/github/ediminished/Deep-Learning/blob/main/VGG_from_Research_Papers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import tqdm

## Model
```https://arxiv.org/pdf/1409.1556.pdf```

In [14]:
# image size: 224 x 224
VGG_types = {
    "VGG-16": [64, 64, "M", 128, 128, "M", 256, 256, 256, "M", 512, 512, 512, "M", 512, 512, 512, "M"] #final shape (N, 512, 7, 7)
} 

class VGG_net(nn.Module):
  def __init__(self, in_channels=3, num_classes=5):
    super().__init__()
    self.in_channels = in_channels
    self.num_classes = num_classes
    self.conv_layers = self.create_conv_layers(VGG_types["VGG-16"])
    self.fcs = nn.Sequential(
        nn.Linear(in_features=512*7*7, out_features=4096),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(in_features=4096, out_features=4096),
        nn.ReLU(),
        nn.Dropout(),
        nn.Linear(in_features=4096, out_features=self.num_classes)
    )
  def forward(self, x):
    x = self.conv_layers(x)
    x = x.view(x.size(0), -1)
    x = self.fcs(x)
    return x

  def create_conv_layers(self, architecture):
    layers = []
    in_channels = self.in_channels

    for x in architecture:
      if type(x) == int:
        out_channels = x
        layers += [nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),nn.BatchNorm2d(out_channels), nn.ReLU()]
        in_channels = x
      elif x == "M":
        layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
    return nn.Sequential(*layers)

model = VGG_net(in_channels=3, num_classes=5)
x = torch.randn(64, 3, 224, 224)
print(model(x).shape)


torch.Size([64, 5])
